<a href="https://colab.research.google.com/github/hsyni/game-character-DCGAN/blob/main/PortraitToCharacterDCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install lpips

Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 111.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 841.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB

In [ ]:
import tensorflow as tf
import torch

print("🔹 TensorFlow GPU Algıladı mı?:", tf.config.list_physical_devices('GPU'))
print("🔹 PyTorch CUDA Kullanılıyor mu?:", torch.cuda.is_available())
print("🔹 PyTorch CUDA Sürümü:", torch.version.cuda)

🔹 TensorFlow GPU Algıladı mı?: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
🔹 PyTorch CUDA Kullanılıyor mu?: True
🔹 PyTorch CUDA Sürümü: 11.8


In [ ]:
!pip uninstall -y torch torchvision torchaudio

Found existing installation: torch 2.5.1+cu118
Uninstalling torch-2.5.1+cu118:
  Successfully uninstalled torch-2.5.1+cu118
Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Successfully uninstalled torchvision-0.20.1+cu124
Found existing installation: torchaudio 2.5.1+cu124
Uninstalling torchaudio-2.5.1+cu124:
  Successfully uninstalled torchaudio-2.5.1+cu124


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/triton-3.2.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.7/848.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.0 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.18 requires torch<2.6,>=1.10, 

In [ ]:
# 📌 Gerekli kütüphaneleri yükle
import os
import cv2
import numpy as np
import tensorflow as tf
import torch
import lpips
from tensorflow.keras.applications import VGG19
from matplotlib import pyplot as plt

# 📌 Google Drive Bağlantısı
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 📌 Model ve Dosya Yolları
generator_path = "/content/drive/MyDrive/YapayZeka/GAN/data/5306F_7151M_Dataset/12457_Total/training_results/generator_weights_epoch_50.weights.h5"
latent_vector_path = "/content/drive/MyDrive/YapayZeka/latent_vector.npy"
input_image_path = "/content/portrait.jpg"
output_anime_path = "/content/generated_anime_image.png"

# 📌 DCGAN Generator Modelini Yükleme
def build_generator(latent_dim):
    noise = tf.keras.Input(shape=(latent_dim,), name="generator_noise_input")
    x = tf.keras.layers.Dense(8 * 8 * 512, use_bias=False)(noise)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Reshape((8, 8, 512))(x)

    filters = [256, 128, 64]
    for f in filters:
        x = tf.keras.layers.Conv2DTranspose(f, kernel_size=5, strides=2, padding="same", use_bias=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(0.2)(x)

    x = tf.keras.layers.Conv2DTranspose(3, kernel_size=5, strides=2, padding="same", activation="tanh")(x)
    return tf.keras.Model(noise, x, name="generator")

LATENT_DIM = 100
generator = build_generator(LATENT_DIM)

# 📌 Ağırlıkları Yükle
try:
    generator.load_weights(generator_path)
    print("✅ Ağırlıklar başarıyla yüklendi!")
except Exception as e:
    raise RuntimeError(f"❌ Ağırlıklar yüklenirken bir hata oluştu: {str(e)}")

# 📌 **Anime Feature Extractor (DCGAN İçin Optimize Edildi)**
def build_feature_extractor():
    base_model = VGG19(weights="imagenet", include_top=False)
    base_model.trainable = False
    selected_layers = [
        base_model.get_layer("block1_conv2").output,  # Daha erken katmanları kullanarak anime stiline daha uygun hale getirdik
        base_model.get_layer("block2_conv2").output,
        base_model.get_layer("block3_conv2").output
    ]
    return tf.keras.Model(inputs=base_model.input, outputs=selected_layers)

feature_extractor = build_feature_extractor()
feature_extractor.summary()

# 📌 **LPIPS Loss Fonksiyonu**
loss_fn = lpips.LPIPS(net='vgg').to(torch.device("cuda"))

def compute_lpips_loss(image1, image2):
    image1_torch = torch.tensor(image1.numpy()).permute(0, 3, 1, 2).to(torch.float32).to("cuda")
    image2_torch = torch.tensor(image2.numpy()).permute(0, 3, 1, 2).to(torch.float32).to("cuda")
    lpips_loss = loss_fn(image1_torch, image2_torch).mean()
    return lpips_loss

# 📌 **Latent Vektör Optimizasyonu**
def optimize_latent_vector(generator, target_image, latent_dim, steps=7000, learning_rate=0.00005):
    latent_vector = tf.Variable(tf.random.normal(shape=(1, latent_dim), dtype=tf.float32), trainable=True)
    optimizer = tf.keras.optimizers.AdamW(learning_rate)

    target_features = feature_extractor(tf.image.resize(target_image, (128, 128)))
    loss_history = []

    for step in range(steps):
        with tf.GradientTape() as tape:
            generated_image = generator(latent_vector)
            generated_features = feature_extractor(tf.image.resize(generated_image, (128, 128)))

            perceptual_loss = sum([
                tf.reduce_mean(tf.square(tf.stop_gradient(t) - g))
                for t, g in zip(target_features, generated_features)
            ])

            lpips_loss = compute_lpips_loss(tf.image.resize(target_image, (128, 128)), tf.image.resize(generated_image, (128, 128)))
            loss = perceptual_loss + 0.3 * lpips_loss  # 🔥 LPIPS ağırlığını düşürdük

        grads = tape.gradient(loss, [latent_vector])
        optimizer.apply_gradients(zip(grads, [latent_vector]))

        loss_history.append(loss.numpy())

        if step % 500 == 0:
            print(f"🔥 Adım {step}/{steps}, Kayıp: {loss.numpy()}")

    np.save(latent_vector_path, latent_vector.numpy())  # 🔥 Latent vektörü kaydet
    return latent_vector.numpy(), loss_history

# 📌 **Görüntü Üretme**
def generate_and_save_anime(generator, latent_vector, output_path):
    anime_image = generator.predict(latent_vector)
    anime_image = (anime_image[0] + 1) * 127.5
    anime_image = np.clip(anime_image, 0, 255).astype(np.uint8)

    plt.imshow(cv2.cvtColor(anime_image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

    cv2.imwrite(output_path, cv2.cvtColor(anime_image, cv2.COLOR_RGB2BGR))
    print(f"✅ Anime tarzı görüntü başarıyla kaydedildi: {output_path}")

# 📌 **Sürecin Çalıştırılması**
original_image = cv2.imread(input_image_path)
if original_image is not None:
    print("✅ Giriş görüntüsü başarıyla yüklendi.")
    preprocessed_image = (original_image / 127.5) - 1.0
    preprocessed_image = tf.expand_dims(preprocessed_image, axis=0)

    optimized_latent_vector, loss_history = optimize_latent_vector(generator, preprocessed_image, LATENT_DIM)
    generate_and_save_anime(generator, optimized_latent_vector, output_anime_path)
else:
    print("❌ Giriş görüntüsü yüklenemedi.")


Mounted at /content/drive


RuntimeError: ❌ Ağırlıklar yüklenirken bir hata oluştu: A total of 7 objects could not be loaded. Example error message for object <Conv2DTranspose name=conv2d_transpose_8, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(5, 5, 256, 512), Received: value.shape=(5, 5, 512, 512). Target variable: <Variable path=conv2d_transpose_8/kernel, shape=(5, 5, 256, 512), dtype=float32, value=[[[[ 6.22888282e-03 -1.67163294e-02 -1.67883039e-02 ... -1.24410298e-02
     1.21840518e-02 -1.53355114e-02]
   [-4.14511282e-03 -7.46158138e-03 -3.59964836e-03 ... -1.39244553e-02
    -5.09580690e-03 -6.59135450e-03]
   [-1.10118557e-02  1.14883259e-02  1.28029268e-02 ... -5.16857766e-03
    -1.42055284e-02  3.10651027e-03]
   ...
   [ 8.94269347e-03 -6.09705225e-04 -1.41174924e-02 ... -6.36001863e-03
     1.38629228e-04  1.10772550e-02]
   [-9.46686510e-03  3.12786922e-03  1.17697734e-02 ...  8.03835690e-03
    -1.3970715